In [ ]:
# ----------------------------------------
# 1. Imports
# ----------------------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# ----------------------------------------
# 2. Dataset Paths (Your Exact Structure)
# ----------------------------------------
train_path = "/kaggle/input/chest-xray-pneumonia/chest_xray/train"
val_path   = "/kaggle/input/chest-xray-pneumonia/chest_xray/val"
test_path  = "/kaggle/input/chest-xray-pneumonia/chest_xray/test"

In [ ]:
# ----------------------------------------
# 3. Data Generators
# ----------------------------------------
img_size = 224
batch_size = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="binary",
    color_mode="grayscale"
)

val_data = val_test_gen.flow_from_directory(
    val_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="binary",
    color_mode="grayscale",
    shuffle=False
)

test_data = val_test_gen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="binary",
    color_mode="grayscale",
    shuffle=False
)

In [ ]:
# ----------------------------------------
# 4. Model: EfficientNetB0
# ----------------------------------------
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(img_size, img_size, 3)
)

inputs = layers.Input(shape=(img_size, img_size, 1))
x = layers.Conv2D(3, (3,3), padding="same")(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
# ----------------------------------------
# 5. Training
# ----------------------------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# ----------------------------------------
# 6. Save model for deployment
# ----------------------------------------
model.save("pneumonia_model.h5")
